# Getting the Data Ready
This notebook is for cleaning up data, generate more columns, change data type, etc.


In [120]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [121]:
#read data, specify index so it's easier to join and search using loc
path = '../../../data/new-york-city-airbnb-open-data/'
listings_csv = os.path.join(path,'selected_columns_listings.csv')
listings = pd.read_csv(listings_csv)

In [122]:
#to get all columns, set option
pd.set_option('display.max_columns', 107)
#to get text with no truncation
pd.set_option('display.max_colwidth', -1)

In [123]:
#shape
listings.shape

(50796, 40)

In [124]:
listings.columns

Index(['id', 'host_since', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_verifications', 'host_identity_verified',
       'neighbourhood_group_cleansed', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'number_of_reviews', 'number_of_reviews_ltm', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'cancellation_policy',
       'reviews_per_month'],
      dtype='object')

# Dealing with null values
Fill with 0, dropping null, fill with avg

## Dropping null rows
Dropping na when rec counts as  there are not too many


In [131]:
Drop_na = ['host_since', 'host_listings_count', 'host_identity_verified', 'host_is_superhost']
listings = listings.dropna(subset=Drop_na)


## Fill null rows with constat value: zero
 or zero ($0.00) variance

In [132]:
#fill with 0
Fill_with_zero = ['bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'review_scores_accuracy',
'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

for column in Fill_with_zero:
    listings[column] = listings[column].replace(np.nan, '0')


In [133]:
#fill with $0.00
Fill_with_zero = ['security_deposit', 'cleaning_fee']

for column in Fill_with_zero:
    listings[column] = listings[column].replace(np.nan, '$0.00')
    

## Fill with average

In [134]:
#host_response_time will be converted to 1,2,3,4. So 2.5 is the middle value
listings['host_response_time'] = listings['host_response_time'].replace(np.nan, '2.5')
#host_acceptance_rate avg 50%
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].replace(np.nan, '50%')


# Put aside features
May add thise features later: 'first_review', 'host_response_rate', 'host_acceptance_rate'
 

# Array data type - reformatting
This step is creating multiple boolean columns to be used in the model
Thi step is no longer needed, creating too many features and seems useless

In [166]:
#function to remove some characters, clean up value
def clean_array(row):
    row = row.str.replace('[', '')
    row = row.str.replace(']', '')
    row = row.str.replace('{', '')
    row = row.str.replace('}', '')
    row = row.str.replace("'", '')
    row = row.str.replace('"', '')
    return row

In [85]:
#array_columns = ['host_verifications', 'amenities']
array_columns = ['host_verifications']
#apply function
#array_columns_clean = listings[array_columns].apply(lambda col: clean_array(col))
array_columns_clean = listings[array_columns].apply(clean_array, axis=1)

# Create new columns based on unique value in the array
array_bool_columns = []

for col in listings[array_columns].columns:
    colname = array_columns_clean[col].str.get_dummies(sep=',');
    colname = colname.add_prefix(col + '_');
    array_bool_columns.append(colname);

#add to the listing
listings = listings.join(array_bool_columns)

#drop original columns
listings = listings.drop(array_columns,axis = 1)

#print all addtional columns
#array_bool_columns

In [86]:
#array_columns = ['host_verifications', 'amenities']
array_columns = ['amenities']
#apply function
#array_columns_clean = listings[array_columns].apply(lambda col: clean_array(col))
array_columns_clean = listings[array_columns].apply(clean_array, axis=1)


# Create new columns based on unique value in the array
array_bool_columns = []

for col in listings[array_columns].columns:
    colname = array_columns_clean[col].str.get_dummies(sep=',');
    colname = colname.add_prefix(col + '_');
    array_bool_columns.append(colname);

#add to the listing
listings = listings.join(array_bool_columns)

#drop original columns
listings = listings.drop(array_columns,axis = 1)

#print all addtional columns
#array_bool_columns

# Currency conversion
This will remove dollars sign, comma and then convert to float

In [136]:
#function
def fix_currency(row):
    row = row.replace(',', '')
    row = row.replace('$', '')
    return row

In [137]:
#Currency_columns = ['extra_people', 'price', 'cleaning_fee', 'security_deposit' ]
#also convert to float

# for col in Currency:
#     listings[col] = listings.apply(fix_currency, axis=1)

colname = 'extra_people'
listings[colname] = listings[colname].apply(lambda col: fix_currency(col)).astype(float)

colname = 'price'
listings[colname] = listings[colname].apply(lambda col: fix_currency(col)).astype(float)

colname = 'cleaning_fee'
listings[colname] = listings[colname].apply(lambda col: fix_currency(col)).astype(float)

colname = 'security_deposit'
listings[colname] = listings[colname].apply(lambda col: fix_currency(col)).astype(float)


In [141]:
def cleaning_fee_yes_no (row):
    if row['cleaning_fee'] > 0.00:
        return '1'
    else:
        return '0'

In [142]:
listings['cleaning_fee_yes_no'] = listings.apply(cleaning_fee_yes_no, axis=1)

In [143]:
listings.groupby('cleaning_fee_yes_no').agg({'price':'mean'})


,price
cleaning_fee_yes_no,
0,201.852941
1,146.563659


In [95]:
def security_deposit_yes_no (row):
    if row['security_deposit'] > 0.00:
        return '1'
    else:
        return '0'

In [145]:
listings['security_deposit_yes_no'] = listings.apply(security_deposit_yes_no, axis=1)

In [146]:
listings.groupby('security_deposit_yes_no').agg({'price':'mean'})

,price
security_deposit_yes_no,
0,144.407500
1,169.972002


In [147]:
def extra_people_yes_no (row):
    if row['extra_people'] > 0.00:
        return '1'
    else:
        return '0'

In [148]:
listings['extra_people_yes_no'] = listings.apply(extra_people_yes_no, axis=1)

In [149]:
listings.groupby('extra_people_yes_no').agg({'price':'mean'})

,price
extra_people_yes_no,
0,171.541619
1,144.059607


In [101]:
listings.groupby('accommodates').agg({'cleaning_fee':'mean', 'security_deposit':'mean', 'price':'mean'}).sort_values(by='accommodates',ascending=False)

,cleaning_fee,security_deposit,price
accommodates,,,
22,270.000000,1000.000000,1569.000000
20,270.000000,999.000000,1479.000000
19,240.000000,1000.000000,1369.000000
16,190.991071,414.017857,683.544643
15,185.562500,484.375000,572.125000
14,170.230769,341.153846,404.269231
13,199.166667,954.166667,773.833333
12,179.321839,447.241379,488.264368
11,143.615385,449.871795,346.794872


In [155]:
#dropping records with certain property type, mostly hotels and records with bad price (0 or more than 2000)
filt = (~listings['property_type'].isin (['Aparthotel','Bed and breakfast','Boutique hotel',' Hostel', 'Hotel', 'Resort', 'Serviced apartment'])) & (listings['price'] > 0) & (listings['price'] < 2000)

#filt = (listings['price'] < 1000) & (listings['price'] > 20)
listings['property_type'][filt].count()

28444

In [156]:
#new df with 
listings = listings[filt]

# Category conversion
Use one hot encoding

In [157]:
# Get one hot encoding of column neighbourhood_group_cleansed
one_hot_encoding_columns = ['neighbourhood_group_cleansed', 'property_type', 'room_type', 'cancellation_policy', 'bed_type']
listings = pd.get_dummies(data=listings, columns=one_hot_encoding_columns)

# Boolean conversion
Converting the value to 1 and 0 from 't' and 'f'.

In [158]:
#function to replace
def convert_bool(row):
    row = row.replace('f', '0')
    row = row.replace('t', '1')
    return row

In [159]:
# #update rows and convert to boo
Boolean_columns = ['host_is_superhost', 'is_location_exact', 'instant_bookable', 'host_identity_verified']

for column in Boolean_columns:
    listings[column] = listings[column].apply(lambda col: convert_bool(col)).astype(int)


# String conversion
Needs to convert to numbers/float

In [160]:
#function to replace
def convert_string_to_int(row):
    row = row.replace('within an hour', '1')
    row = row.replace('within a few hours', '2')
    row = row.replace('within a day', '3')
    row = row.replace('a few days or more', '4')
    return row

In [161]:
colname = ['host_response_time']

listings[colname] = listings[colname].apply(lambda col: convert_string_to_int(col)).astype(float)

In [162]:
listings.groupby('host_response_time')['host_response_time'].count()

host_response_time
1.0    16282
2.0    4549 
2.5    4067 
3.0    2947 
4.0    599  
Name: host_response_time, dtype: int64

# Date conversion
Convert to date then to ordinal

In [163]:
#conver to date first
listings['host_since'] = pd.to_datetime(listings['host_since'])

listings['host_since'] = listings['host_since'].apply(lambda x: x.toordinal())



In [164]:
listings['host_since'].head()

1    733294
2    733383
3    733440
5    733441
6    733469
Name: host_since, dtype: int64

In [165]:
listings.head(1)

,id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_verifications,host_identity_verified,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,last_review_days_ago,cleaning_fee_yes_no,security_deposit_yes_no,extra_people_yes_no,neighbourhood_group_cleansed_Bronx,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,property_type_Apartment,property_type_Barn,property_type_Boat,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Casa particular (Cuba),property_type_Castle,property_type_Cave,property_type_Condominium,property_type_Cottage,property_type_Earth house,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_House,property_type_Houseboat,property_type_Island,property_type_Lighthouse,property_type_Loft,property_type_Other,property_type_Tent,property_type_Tiny house,property_type_Townhouse,property_type_Treehouse,property_type_Villa,property_type_Yurt,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
1,2595,733294,3.0,87%,38%,0,6.0,"['email', 'phone', 'reviews', 'offline_government_id', 'kba', 'selfie', 'government_id', 'identity_manual', 'work_email']",1,40.75362,-73.98377,0,2,1,0,1,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid parking off premises"",""Free street parking"",Heating,""Family/kid friendly"",""Smoke detector"",""Carbon monoxide detector"",""Fire extinguisher"",Essentials,""Lock on bedroom door"",""Hair dryer"",Iron,""Laptop friendly workspace"",""Self check-in"",Keypad,""Private living room"",Bathtub,""Hot water"",""Bed linens"",""Extra pillows and blankets"",""Ethernet connection"",""Coffee maker"",Refrigerator,""Dishes and silverware"",""Cooking basics"",Oven,Stove,""Luggage dropoff allowed"",""Long term stays allowed"",""Cleaning before checkout""}",225.0,350.0,95.0,1,0.0,48,5,2009-11-21,2019-11-04,94,9,9,10,10,10,9,0,0.38,131.0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [169]:
listings.columns.values

array(['id', 'host_since', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_verifications', 'host_identity_verified', 'latitude',
       'longitude', 'is_location_exact', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'amenities', 'price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people',
       'number_of_reviews', 'number_of_reviews_ltm', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_per_month',
       'last_review_days_ago', 'cleaning_fee_yes_no',
       'security_deposit_yes_no', 'extra_people_yes_no',
       'neighbourhood_group_cleansed_Bronx',
       'neighbourhood_group_cleansed_Brooklyn',
       'neighbourhood_group_cleansed_Manhattan',


# Export to CSV
Clean data for model to use

In [170]:
#export columns left to csv to be used with second notebook
path = '../../../data/new-york-city-airbnb-open-data/'

listings_csv = os.path.join(path,'model_columns_listings.csv')
listings.to_csv(listings_csv)